In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# cd ~/automl_for_competition/autox/data
# scp -r kaggle_springleaf/ caihengxing@model-gpu01:~/automl_for_competition/autox/data

In [3]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,9 days 23 hours 6 mins
H2O_cluster_timezone:,Asia/Shanghai
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,1 month and 12 days
H2O_cluster_name:,H2O_from_python_caihengxing_5p2i4b
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,23.65 Gb
H2O_cluster_total_cores:,40
H2O_cluster_allowed_cores:,40
H2O_cluster_status:,"locked, healthy"


In [5]:
import os
os.listdir("../autox/data/")

['kaggle_house_price',
 'stumbleupon',
 'kaggle_ieee',
 'zhidemai',
 'titanic',
 'zhidemai_competition_0715',
 '.ipynb_checkpoints',
 'kaggle_springleaf',
 'kaggle_tabular_aug_2021']

In [6]:
data_name = 'stumbleupon'

In [7]:
train_path = f'../autox/data/{data_name}/train.csv'
test_path = f'../autox/data/{data_name}/test.csv'

In [8]:
# Load data into H2O
train = h2o.import_file(train_path)
test  = h2o.import_file(test_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
train.head()

url,urlid,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html,4042,business,0.789131,2.05556,0.676471,0.205882,0.0470588,0.0235294,0.443783,0,0,0.0907738,0,0.245831,0.00388349,0,1,24,0,5424,170,8,0.152941,0.0791296,0
http://www.popsci.com/technology/article/2012-07/electronic-futuristic-starting-gun-eliminates-advantages-races,8471,recreation,0.574147,3.67797,0.508021,0.28877,0.213904,0.144385,0.468649,0,0,0.0987074,0,0.20349,0.0886525,0,1,40,0,4973,187,9,0.181818,0.125448,1
http://www.menshealth.com/health/flu-fighting-fruits?cm_mmc=Facebook-_-MensHealth-_-Content-Health-_-FightFluWithFruit,1164,health,0.996526,2.38288,0.562016,0.321705,0.120155,0.0426357,0.525448,0,0,0.0724479,0,0.226402,0.120536,0,1,55,0,2240,258,11,0.166667,0.0576132,1
http://www.dumblittleman.com/2007/12/10-foolproof-tips-for-better-sleep.html,6684,health,0.801248,1.5431,0.4,0.1,0.0166667,0,0.480725,0,0,0.0958606,0,0.265656,0.035343,0,0,24,0,2737,120,5,0.0416667,0.100858,1
http://bleacherreport.com/articles/1205138-the-50-coolest-jerseys-you-didnt-know-existed?show_full=,9006,sports,0.719157,2.67647,0.5,0.222222,0.123457,0.0432099,0.446143,0,0,0.0249084,0,0.228887,0.0504732,0,1,14,0,12032,162,10,0.0987654,0.0825688,0
http://www.conveniencemedical.com/genital-herpes-home.php,7018,?,nan,119,0.745455,0.581818,0.290909,0.0181818,0.434639,0,0,0.0198413,0,0.2983,0.0386364,-1,0,12,-1,4368,55,3,0.0545455,0.0873563,0
http://gofashionlane.blogspot.tw/2012/06/american-wild-child.html,8685,arts_entertainment,0.22111,0.77381,0.215054,0.0537634,0.0430108,0.0430108,0.579596,0,0,0.0395683,0,0.218978,0.311377,0,0,21,0,1287,93,3,0.548387,0.0643275,1
http://www.insidershealth.com/article/racing_for_recovery/3471,3402,?,nan,1.88333,0.719697,0.265152,0.113636,0.0151515,0.499348,0,0,0.026616,0,0.173746,0.0258303,-1,0,5,-1,27656,132,4,0.0681818,0.148551,0
http://www.valetmag.com/the-handbook/features/31-days/index.php?index1_middle_feature,477,?,nan,0.471503,0.190722,0.0360825,0,0,0.383199,0,0,0.0217054,0,0.114962,1.13665,0,0,17,0,2471,194,7,0.64433,0.125,1
http://www.howsweeteats.com/2010/03/24/cookies-and-cream-brownies/,6731,?,nan,2.41011,0.469325,0.101227,0.0184049,0.00306749,0.465859,0,0,0.012,0,0.205117,0.206262,-1,1,14,-1,11459,326,4,0.236196,0.0944123,1


In [10]:
Id = "urlid"
y = "label"

In [11]:
aml2 = H2OAutoML(max_runtime_secs = 7200, seed = 1, project_name = data_name)
aml2.train(y = y, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [12]:
pred = aml2.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [13]:
sub = pd.read_csv(test_path)
sub = sub[[Id]]

In [14]:
pred = h2o.as_list(pred)

In [15]:
sub[y] = pred['predict']

In [16]:
sub['label'] = sub['label'].clip(0,1)

In [17]:
sub.to_csv(f"./h2o_sub_{data_name}.csv", index = False)